The microsatellite dataset is transformated into a binary format applying one hot encoding, while also incorporating infromation on missing values and heterozygosity for each locus.

Data pre-processing:

In [ ]:
# Load libraries

import pandas as pd
import numpy as np

In [ ]:
# Load dataset
microsat_data = pd.read_excel('pine_ML.xlsx')

microsat_data.head()

In [ ]:
# Store location data
location_data = microsat_data['Location']

#Remove 'Plant' and 'Location' columns
microsat_data = microsat_data.drop('Plant', axis= 1)
microsat_data = microsat_data.drop('Location', axis = 1)

microsat_data

In [ ]:
#Make a list with the column names
locus_columns = list(microsat_data.columns)

print(locus_columns)

In [ ]:
# Create a list of allele pairs by grouping every two alleles
loci_pairs = [(locus_columns[i], locus_columns[i+1]) for i in range(0, len(locus_columns), 2)]

print(loci_pairs)

In [ ]:
# Find unique repeat lengths for each locus

unique_lengths_dict = {}  # Make a dictionary to store unique lengths for each locus

for allele1, allele2 in loci_pairs:
    combined_unique_lengths = pd.concat([microsat_data[allele1], microsat_data[allele2]]).unique()   # Combine the lengths of the two alleles of each locus and keep only the unique lengths.

    # Extract the name of the locus
    locus_name = allele1.split('_')[0]

    unique_lengths_dict[locus_name] =list(combined_unique_lengths)

# Print the dictionary with unique lengths for each locus
for locus_name, unique_lengths in unique_lengths_dict.items():
    print(f"Locus: {locus_name}, Unique Lengths: {unique_lengths}")

Extra columns representing Heterozygosity and Missing Values in a binary format are included.

In [ ]:
# Create a new column for heterozygosity.

for allele1, allele2 in loci_pairs:
    # Extract the locus name
    locus_name = allele1.split('_')[0]

    # Create a new column for heterozygosity (e.g., 'KU85_het')
    het_column = f"{locus_name}_het"

    # Compare the two alleles for each individual
    # If alleles are different (heterozygous), assign 1; otherwise, assign 0 (homozygous)
    microsat_data[het_column] = (microsat_data[allele1] != microsat_data[allele2]).astype(int)

# Check the modified dataframe
print(microsat_data.head())

In [ ]:
# Missing values indicator - Create two new columns (missing_a, missing_b)

for locus_name in unique_lengths_dict.keys():  # Use unique lengths dictionary to get locus names

    # Check missing values for allele 'a'
    microsat_data[f"{locus_name}_a_missing"] = microsat_data[f"{locus_name}_a"].isna().astype(int)

    # Check missing values for allele 'b'
    microsat_data[f"{locus_name}_b_missing"] = microsat_data[f"{locus_name}_b"].isna().astype(int)

print(microsat_data)

One Hot Encoding:

In [ ]:
# Create a list to store the one-hot encoded columns as individual DataFrames
encoded_columns = []

# Loop over each locus and its unique lengths
for locus_name, unique_lengths in unique_lengths_dict.items():

    # Loop over each unique length (allele size) for this locus
    for unique_length in unique_lengths:

        # Create column names for one-hot encoding (for both alleles)
        col_name_a = f"{locus_name}_a-{unique_length}"
        col_name_b = f"{locus_name}_b-{unique_length}"

        # One-hot encode the allele a: 1 if it matches the unique length, else 0
        col_a = (microsat_data[f"{locus_name}_a"] == unique_length).astype(int).rename(col_name_a)

        # One-hot encode the allele b: 1 if it matches the unique length, else 0
        col_b = (microsat_data[f"{locus_name}_b"] == unique_length).astype(int).rename(col_name_b)

        # Append the two columns as individual DataFrames to the list
        encoded_columns.append(col_a)
        encoded_columns.append(col_b)

# Concatenate all columns in the list into a new DataFrame
df_onehot = pd.concat(encoded_columns, axis=1)

df_onehot

In [ ]:
# Remove nan columns from the dataframe

# Drop columns where the column name ends with 'nan'
df_onehot = df_onehot.drop(df_onehot.filter(regex='nan$').columns, axis=1)

# Display the updated dataframe
df_onehot

In [ ]:
# Add heterozygosity columns from microsat_data to the one-hot encoded data
df_het = microsat_data.filter(like='_het')  # Heterozygosity columns

# Add missing value columns from microsat_data to the one-hot encoded data
df_missing = microsat_data.filter(like='_missing')  # Missing values columns

# Combine all columns
final_df = pd.concat([df_onehot, df_het, df_missing], axis=1)

final_df

In [ ]:
print(final_df.size)
print(final_df.shape)